This notebook will mainly be used for capstone project

In [1]:
import pandas as pd
import numpy as np


<font color='blue'>Read table from wikipedia page using pandas</font>

In [2]:
page= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(page, attrs={"class":"wikitable"})
table = table[0]

In [3]:
table.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<font color='blue'>Drop all the rows with 'Not assigned' in 'Borough' column</font>

In [4]:
filt1 = table['Borough'] == 'Not assigned'
table.drop(index = table[filt1].index, inplace = True)
table.reset_index(drop=True, inplace = True)
table.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Replace the 'Not assigned' value with NaN, then fillna with left column 'Borough'

In [5]:
table = table.replace({
    'Neighborhood':'Not assigned'
    }, np.NaN)
table = table.fillna(method='ffill', axis=1)
table.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
table.shape

(103, 3)